In [0]:
! pip install kashgari
! pip install pandas
! pip install tensorflow==1.15.0

In [3]:



#請載資料夾內的rbt3_model模型並解壓縮，才能用這個預測菜名的模型

import pandas as pd
import kashgari
import re


#使用這個目錄下的模型
model_path = 
loaded_model = kashgari.utils.load_model(model_path+'\rbt3_model')

#定義類別(要用裡面的Get方法才能唷~~)
class Get_dishes:

    global loaded_model

    def __init__(self,df):
        self.D = df

    def cut_text(self,text, lenth):
        textArr = re.findall('.{' + str(lenth) + '}', text)
        textArr.append(text[(len(textArr) * lenth):])
        return textArr


    def extract_labels(self,text, ners):
        ner_reg_list = []
        if ners:
            new_ners = []
            for ner in ners:
                new_ners += ner;
            for word, tag in zip([char for char in text], new_ners):
                if tag != 'O':
                    ner_reg_list.append((word, tag))

        # 輸出模型的NER识别结果
        labels = {}
        if ner_reg_list:
            for i, item in enumerate(ner_reg_list):
                if item[1].startswith('B'):
                    label = ""
                    end = i + 1
                    while end <= len(ner_reg_list) - 1 and ner_reg_list[end][1].startswith('I'):
                        end += 1

                    ner_type = item[1].split('-')[1]
                    if ner_type not in labels.keys():
                        labels[ner_type] = []

                    label += ''.join([item[0] for item in ner_reg_list[i:end]])
                    labels[ner_type].append(label)

        return labels

    #建立一個能讓dataframe中的評論欄位,能預測菜單的函式
    def get_name(self,sentence):

        text_1 = self.cut_text(sentence, 100)
        ners = loaded_model.predict([[char for char in text] for text in text_1])
        labels = self.extract_labels(sentence, ners)
        x = labels.get("1","")
        return ",".join(x)

    #修改dish欄位，放入預測的菜單
    def Get(self):
        self.D["dish"] = self.D["text"].apply(self.get_name)
        return self.D

/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:22: UserWarning: bert4keras.bert has been renamed as bert4keras.models.
  warnings.warn('bert4keras.bert has been renamed as bert4keras.models.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:23: UserWarning: please use bert4keras.models.
  warnings.warn('please use bert4keras.models.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:44: UserWarning: bert4keras.tokenizer has been renamed as bert4keras.tokenizers.
  warnings.warn('bert4keras.tokenizer has been renamed as bert4keras.tokenizers.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:45: UserWarning: please use bert4keras.tokenizers.
  warnings.warn('please use bert4keras.tokenizers.')
